In [114]:
from pymongo import MongoClient
import pandas as pd
import re

In [115]:
# Importo archivo después de geoindex en Mongo Compass

client = MongoClient('mongodb://localhost:27017/')
db = client.germany_after_geoindex


In [116]:
comps = db.germany_after_geoindex.find({
    'offices.country_code': 'DEU'
        })

In [117]:
# Creo el DataFrame

germany_offices = pd.DataFrame(comps)
germany_offices.head()

,_id,category_code,city,description,founded_year,geo,lat,lng,name,number_of_employees,offices,overview,tag_list,total_money_raised
0,5d2760d235ebd4b66763581c,web,Berlin,Free Facebook video chat app.,2012.0,"{'type': 'Point', 'coordinates': [13.4109071, ...",52.501345,13.410907,headr,8.0,"[{'description': '', 'address1': 'Ritterstr. 1...","<p><a href=""http://vive.me/"" title=""vive"" rel=...",,$0
1,5d2760d235ebd4b66763581d,web,Muenster,"Syncs Contacts, calender and tasks",2010.0,"{'type': 'Point', 'coordinates': [7.6267552, 5...",51.963009,7.626755,fruux,NaN,"[{'description': '', 'address1': '', 'address2...",<p>fruux solves the problem of working with co...,"sync, synchronize, contacts, calendars, events...",$0
2,5d2760d235ebd4b66763581e,enterprise,Potsdam,Global Publishing Platform,2009.0,"{'type': 'Point', 'coordinates': [13.0673441, ...",52.405558,13.067344,printium,3.0,"[{'description': 'HQ', 'address1': 'Behlertstr...",<p>printium&#8217;s vision is to provide all p...,"print, publishing, community, global, directory",$0
3,5d2760d235ebd4b66763581f,network_hosting,Oldenburg,Technology wiki,2009.0,"{'type': 'Point', 'coordinates': [8.200653, 53...",53.149670,8.200653,youserbase,2.0,"[{'description': 'Headquarters', 'address1': '...",<p>youserbase is a technology knowledge platfo...,"germany, oldenburg, wiki",$0
4,5d2760d235ebd4b667635820,web,Karlsruhe,Free Online Dating,2009.0,"{'type': 'Point', 'coordinates': [8.4649954, 4...",48.997425,8.464995,FindandSmile,2.0,"[{'description': '705 West GmbH', 'address1': ...",<p>Find&amp;Smile introduces Free Online Datin...,"dating, free, online-dating, dating-site, free...",$0


In [122]:
# Creo una clase 'Finder' en la que incluyo todas las funciones para localizar las oficinas
# de otras empresas en distintos radios y aplicarles puntuaciones en función de distintos
# criterios: distancia, antigüedad, fondos, número de empleados y categoría, para luego
# sumarlos y asignar una puntuación total a cada ubicación

class Finder():
    def __init__(self, database):
        self.database = database
        self.radius_list = [2, 4, 6, 8, 10]
        self.radius_points = [
            {"id": 2, "points": 10},
            {"id": 4, "points": 8},
            {"id": 6, "points": 6},
            {"id": 8, "points": 4},
            {"id": 10, "points": 2}]


    def find_near(self, geopoint, radio_max_meters = 1):
        return self.database.germany_offices.find({
            "geo": {
                "$near": {
                    "$geometry": geopoint,
                    "$maxDistance": radio_max_meters,
                }
            }
        })


    def get_total_score(self, database):
        total_score = 0
        office_geo = database['geo']
        for office_geo_p in office_geo:
            for radius in self.radius_list:
                near_companies = self.find_near(office_geo, radius*1000)            
                for near_company in near_companies:
                    score = get_score(near_company, radius)
                    total_score += score
        return {'score': total_score} 
                
                
    def get_radius_points(self, radius):
        return (rad_point for rad_point in self.radius_points if rad_point["id"] == radius)


    def get_founded_year_points(self, year):
        if year >= 2002.0:
            return 10
        else:
            return 0


    def get_money_points(self, money):
        if (r'\M+$') in money:
            return 10
        else:
            return 0


    def get_employees_points(self, number):
        if number < 100:
            return 10
        else:
            return 0
        
    
    def get_category_code(self, code):
        if category_code == ('games_video') or ('web'):
            return 10
        else:
            return 0


    def get_score(self, company, radius):
        return self.get_radius_points(radius) + self.get_founded_year_points(germany_offices['founded_year']) + self.get_money_points(germany_offices['total_money_raised'])+self.get_employees_points(germany_offices['number_of_employees'])+self.get_category_code(germany_offices['category_code'])


# Instancio la clase 'Finder'

localizer = Finder(germany_offices)

scores = germany_offices[['geo']].apply(localizer, result_type='expand', axis=1)
scores.head()

TypeError: ("'Finder' object is not callable", 'occurred at index 0')

In [119]:
# Creo un nuevo DataFrame 'points' agregando la columna anterior 'score' al DataFrame previo 'germany_offices'

points = pd.concat([germany_offices,scores], axis=1)[["name","geo","total_money_raised","founded_year","number_of_employees","category_code"]]
points.head(3)

NameError: name 'scores' is not defined